In [ ]:
import stim
import sys
import os
sys.path.append(os.path.abspath("../src"))
from bb_ions import *
from pysat.examples.rc2 import RC2
from pysat.formula import WCNF


# # Uncomment code(s) of choice:

code = bb5_30_4_5_code() 
# code = bb5_48_4_7_code() 
# code = bb6_72_12_6_code() 
# code = bb6_90_8_10_code() 
# code = bb6_108_code() 
# code = gross_code() 
# code = two_gross_code() 

p = 0.01
memory_basis = 'Z'

circuit = make_BB_circuit(  # see src/bb_ions/circfuncs for explanation of make_BB_circuit inputs
    code,  
    p,  
    errors = uniform_errors(p),
    idle_during = uniform_idling(p),
    num_syndrome_extraction_cycles = 1, 
    memory_basis = memory_basis,
    sequential_gates = True, 
    exclude_opposite_basis_detectors = True,
    reuse_check_qubits = True,  
)

In [ ]:
import stim
circuit = stim.Circuit.from_file("../circuits/relaybp_circuits/circuit=bicycle_bivariate_144_12_12_memory_X,distance=12,rounds=12,error_rate=0.001,noise_model=uniform_circuit,basis=CX,A=x^3+y+y^2,B=y^3+x+x^2.stim")

svg = str(circuit.diagram("timeline-svg"))
with open(f"circuit_diagram.svg", "w", encoding="utf-8") as f: f.write(svg)


3 methods to try find circuit-level distance:

In [ ]:
try:
    print(len(circuit.search_for_undetectable_logical_errors(
        dont_explore_detection_event_sets_with_size_above=30,
        dont_explore_edges_with_degree_above=30,
        dont_explore_edges_increasing_symptom_degree=False,
    )))
except Exception as e:
    print(e)

In [ ]:
try:
    len(circuit.shortest_graphlike_error())
except Exception as e:
    print(e)

In [ ]:
# Generate maxSAT problem of the circuit's distance, that other tools can solve.
with open("problem.wcnf", "w") as file:
    file.write(circuit.shortest_error_sat_problem())

# Solve the problem using python-sat
wcnf = WCNF(from_file="problem.wcnf")

with RC2(wcnf) as rc2:

    sol = rc2.compute()
    for entry in sol:
        if entry > 0:
            print(entry)
    

    print(f"\nd_circ = {rc2.cost}")

In [ ]:
# # Using CorePlus. Supposedly faster but I didn't find that to be so.
# !./CASHWMaxSAT-CorePlus/bin/cashwmaxsatcoreplus -bm -m problem.wcnf